## Discriminator

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import pandas as pd

In [3]:
import random
from collections import defaultdict

In [4]:
import codecs

In [5]:
from sklearn.utils import shuffle

In [6]:
from collections import Counter

In [7]:
import itertools

### load dataset
##### in real model, the dataset for discriminator should be: generated answer: 0, ground truth answer: 1
##### here: lexRank selected sentences: 0, ground truth answer: 1

## Dataloader

In [8]:
import json

mydata = json.load(open('/Users/zhangyiman/Desktop/nfL6.json','r'))

questions = []
bestAnswers = []
NotBestAnswers = []
main_categorys = []
q_id = []

for q_a in mydata:
    questions.append(q_a['question'])
    bestAnswers.append(q_a['answer'])
    NotBestAnswers.append(q_a['nbestanswers'])
    main_categorys.append(q_a['main_category'])
    q_id.append(q_a['id'])
    

In [9]:
#print('question[0]')
print(questions[0])
#print('ground_truth answer for this question')
print(bestAnswers[0])

# use question + bestAnswer as the real data to train the discriminator
# need to write it into real_QA.txt line by line

Why did the U.S Invade Iraq ?
A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.


In [10]:
# sentences candidate

query_id = []
query = []
category = []
sentence_id = []
sentence = []
whether_groundTruth = []
for j in range (0,len(NotBestAnswers)):
    # for every question, 
    sentence_candidate = []
    Ground_Truth_sentence = []
    
    # there are len_sen answers for this question
    len_sen = len(NotBestAnswers[j])
    for a in range(0,len_sen):
        str = NotBestAnswers[j][a]
        str_split = str.split('. ')
        # reduce if sentence is null
        for i in range(0,len(str_split)):
            if len(str_split[i])>=2:
                sentence_candidate.append(str_split[i])
                
    #Ground Truth sentences
    str = bestAnswers[j]
    str_split = str.split('. ')
    for i in range(0,len(str_split)):
        Ground_Truth_sentence.append(str_split[i])
        
    #Whether ground truth
    for i in range(0,len(sentence_candidate)):
        if sentence_candidate[i] not in Ground_Truth_sentence:
            Ground_Truth = 0
        else:
            Ground_Truth = 1
        query_id.append(j)
        query.append(questions[j])
        category.append(main_categorys[j])
        sentence_id.append(i)
        sentence.append(sentence_candidate[i])
        whether_groundTruth.append(Ground_Truth)
    

##### use question + bestAnswer as the real data to train the discriminator
##### need to write it into real_QA.txt line by line

In [11]:
bestAnswers[1]

'Call an area apiarist.  They should be able to help you and would most likely remove them at no charge in exchange for the hive.  The bees have value and they now belong to you.'

In [12]:
len(questions)

87362

In [13]:
with open('real_QA.txt','w') as f:
    
    for i in range(0,70000):
        f.write(questions[i])
        f.write(' ')
        f.write(bestAnswers[i])
        f.write('\n')
f.close

<function TextIOWrapper.close>

## data : positive, negative

In [14]:
positive_examples = []
negative_examples = []

In [15]:
positive_examples = list(codecs.open("real_QA.txt", "r", "utf-8").readlines())
positive_examples = [s.split() for s in positive_examples]

In [16]:
negative_examples = list(codecs.open('generated_QA.txt', "r", "utf-8").readlines())
negative_examples = [s.split() for s in negative_examples]

In [17]:
len(positive_examples)

70021

## use part of data to test the model

In [18]:
positive_examples = positive_examples[:5000]
negative_examples = negative_examples[:5000]

In [19]:
#Pads all sentences to the same length.
padding_word="<PAD/>"

positive_length = max(len(x) for x in positive_examples)
negative_length = max(len(x) for x in negative_examples)
if positive_length > negative_length:
    sequence_length = positive_length
else:
    sequence_length = negative_length
            
padded_positive_examples = []
for i in range(len(positive_examples)):
    sentence = positive_examples[i]
    num_padding = sequence_length - len(sentence)
    new_sentence = sentence + [padding_word] * num_padding
    padded_positive_examples.append(new_sentence)
        
padded_negative_examples = []
for i in range(len(negative_examples)):
    sentence = negative_examples[i]
    num_padding = sequence_length - len(sentence)
    new_sentence = sentence + [padding_word] * num_padding
    padded_negative_examples.append(new_sentence)
        
        
#build_vocab
#Builds a vocabulary mapping from word to index based on the sentences.
sentences = padded_positive_examples + padded_negative_examples
word_counts = Counter(itertools.chain(*sentences))
# Mapping from index to word
vocabulary_inv = [x[0] for x in word_counts.most_common()]
# Mapping from word to index
vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}

_sentences = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])

In [20]:
len(vocabulary)

58158

In [21]:
#print(_sentences[1])

In [22]:
sentences = _sentences

In [28]:
positive_labels = [[0, 1] for _ in positive_examples]
negative_labels = [[1, 0] for _ in negative_examples]
labels = np.concatenate([positive_labels, negative_labels], 0)

In [29]:
a = sentences
b = labels
        
a,b = shuffle(a,b)
        
sentences = a
lables = b

##### generates_QA.txt

In [13]:
import csv

with open('sentence_QA.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['query_id','query','main_category','sentence_id','sentence','whether_groundTruth'])
    for i in range(0,len(query_id)):
        writer.writerow([query_id[i],query[i],category[i],sentence_id[i],sentence[i],whether_groundTruth[i]])

In [14]:
sentence_data = pd.read_csv('sentence_QA.csv')

In [21]:
sentence_selected = defaultdict(list)
for i in range(0,87362):
    if len(sentence_data[sentence_data['query_id']== i]) > 3: 
        sen_num = random.sample(range(len(sentence_data[sentence_data['query_id']== i])),3)
        for j in range(0,3):
            sub = sentence_data[sentence_data['query_id']==i][sentence_data['sentence_id'] ==sen_num[j]]
            sentence_sel = sub['sentence'].values[0]
            sentence_selected[i].append(sentence_sel)
    else :
        if len(sentence_data[sentence_data['query_id']== i]) >= 1 :
            sen_num = random.sample(range(len(sentence_data[sentence_data['query_id']== i])),1)
            for j in range(0,1):
                sub = sentence_data[sentence_data['query_id']==i][sentence_data['sentence_id'] ==sen_num[j]]
                sentence_sel = sub['sentence'].values[0]
                sentence_selected[i].append(sentence_sel)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [22]:
sentence_selected[1][2]

'He then collects the honey in the empty bag, pulls the hive into the bag of coals.'

In [23]:
len(sentence_selected)

87353

In [24]:
neg_examples = []
for i in range(0,len(sentence_selected)):
    str_sen = ''
    str_neg = ''
    for j in range(0,len(sentence_selected[i])):
        str_sen = str_sen + sentence_selected[i][j]
    str_neg = str_neg + questions[i]
    str_neg = str_neg + str_sen
    neg_examples.append(str_neg)

In [25]:
neg_examples[50]

"what is the answer to: P=2L + 2w for w ?If you want a formula that will yield W, it's: (P-2L)/2 = WThat depends on what L and P areperimeter of a rectangle with length L and width w is =2L+2w"

In [26]:
len(neg_examples)

87353

### write negative examples and positive examples into the txt file
In GAN, we need to get the negative examples from the generator. 
Then use two txt file to train the discriminator.

In [27]:
with open('generated_QA.txt','w') as f:
    for i in range(0,len(neg_examples)):
        f.write(neg_examples[i])
        f.write('\n')
f.close()

## load data for Discriminator
#### from positive_file and negative_file

In [8]:
positive_file = 'real_QA.txt'
negative_file = 'generated_QA.txt'

In [9]:
class Dis_dataloader():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])

    def load_train_data(self,positive_file, negative_file):
        # Load data
        positive_examples = []
        negative_examples = []

        positive_examples = list(codecs.open(positive_file, "r", "utf-8").readlines())
        positive_examples = [s.split() for s in positive_examples]
                
        
        negative_examples = list(codecs.open(negative_file, "r", "utf-8").readlines())
        negative_examples = [s.split() for s in negative_examples]
        
        # use part of data to test the model
        positive_examples = positive_examples[:5000]
        negative_examples = negative_examples[:5000]
        
        #Pads all sentences to the same length.
        padding_word="<PAD/>"

        positive_length = max(len(x) for x in positive_examples)
        negative_length = max(len(x) for x in negative_examples)
        if positive_length > negative_length:
            sequence_length = positive_length
        else:
            sequence_length = negative_length
            
        padded_positive_examples = []
        for i in range(len(positive_examples)):
            sentence = positive_examples[i]
            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
            padded_positive_examples.append(new_sentence)
        
        padded_negative_examples = []
        for i in range(len(negative_examples)):
            sentence = negative_examples[i]
            num_padding = sequence_length - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
            padded_negative_examples.append(new_sentence)
        
        self.sentences = padded_positive_examples + padded_negative_examples
        
        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        #############################################################################
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        a = self.sentences
        b = self.labels
        
        a,b = shuffle(a,b)
        
        self.sentences = a
        self.lables = b

        
        #build_vocab
        #Builds a vocabulary mapping from word to index based on the sentences.
        sentences = self.sentences
        word_counts = Counter(itertools.chain(*sentences))
        # Mapping from index to word
        vocabulary_inv = [x[0] for x in word_counts.most_common()]
        # Mapping from word to index
        vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}

        #############################################################################
        self.sentences = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
        
        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)

        self.pointer = 0


    def next_batch(self):
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0


In [10]:
#TOTAL_BATCH = 200
#positive_file = 'save/real_data.txt'
#negative_file = 'save/generator_sample.txt'
#eval_file = 'save/eval_file.txt'

### Discriminator: CNN

In [11]:
# An alternative to tf.nn.rnn_cell._linear function, which has been removed in Tensorfow 1.0.1
# The highway layer is borrowed from https://github.com/mkroutikov/tf-lstm-char-cnn
def linear(input_, output_size, scope=None):
    '''
    Linear map: output[k] = sum_i(Matrix[k, i] * input_[i] ) + Bias[k]
    Args:
    input_: a tensor or a list of 2D, batch x n, Tensors.
    output_size: int, second dimension of W[i].
    scope: VariableScope for the created subgraph; defaults to "Linear".
  Returns:
    A 2D Tensor with shape [batch x output_size] equal to
    sum_i(input_[i] * W[i]), where W[i]s are newly created matrices.
  Raises:
    ValueError: if some of the arguments has unspecified or wrong shape.
  '''

    shape = input_.get_shape().as_list()
    if len(shape) != 2:
        raise ValueError("Linear is expecting 2D arguments: %s" % str(shape))
    if not shape[1]:
        raise ValueError("Linear expects shape[1] of arguments: %s" % str(shape))
    input_size = shape[1]

    # Now the computation.
    with tf.variable_scope(scope or "SimpleLinear"):
        matrix = tf.get_variable("Matrix", [output_size, input_size], dtype=input_.dtype)
        bias_term = tf.get_variable("Bias", [output_size], dtype=input_.dtype)

    return tf.matmul(input_, tf.transpose(matrix)) + bias_term

In [12]:
def highway(input_, size, num_layers=1, bias=-2.0, f=tf.nn.relu, scope='Highway'):
    """Highway Network (cf. http://arxiv.org/abs/1505.00387).
    t = sigmoid(Wy + b)
    z = t * g(Wy + b) + (1 - t) * y
    where g is nonlinearity, t is transform gate, and (1 - t) is carry gate.
    """

    with tf.variable_scope(scope):
        for idx in range(num_layers):
            g = f(linear(input_, size, scope='highway_lin_%d' % idx))

            t = tf.sigmoid(linear(input_, size, scope='highway_gate_%d' % idx) + bias)

            output = t * g + (1. - t) * input_
            input_ = output

    return output


In [13]:
class Discriminator(object):
    '''A CNN for classification: 
    Distinguish between the ground truth answer and the generated answer
    CNN: embedding layer --> convolutional, max pooling-->softmax '''
    
    # parameters
    def __init__(
                self, 
                sequence_length, 
                num_classes, 
                vocab_size,
                embedding_size, 
                filter_sizes, 
                num_filters, 
                l2_reg_lambda=0.0):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        with tf.variable_scope('discriminator'):
            # Embedding layer
            with tf.device('/cpu:0'), tf.name_scope("embedding"):
                self.W = tf.Variable(
                    tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                    name="W")
                self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
                self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)


            # Create a convolution + maxpool layer for each filter size
            pooled_outputs = []
            for filter_size, num_filter in zip(filter_sizes, num_filters):
                with tf.name_scope("conv-maxpool-%s" % filter_size):
                    # Convolution Layer
                    filter_shape = [filter_size, embedding_size, 1, num_filter]
                    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                    b = tf.Variable(tf.constant(0.1, shape=[num_filter]), name="b")
                    conv = tf.nn.conv2d(
                        self.embedded_chars_expanded,
                        W,
                        strides=[1, 1, 1, 1],
                        padding="VALID",
                        name="conv")
                    # Apply nonlinearity
                    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                    # Maxpooling over the outputs
                    pooled = tf.nn.max_pool(
                        h,
                        ksize=[1, sequence_length - filter_size + 1, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name="pool")
                    pooled_outputs.append(pooled)
            
            # Combine all the pooled features
            num_filters_total = sum(num_filters)
            self.h_pool = tf.concat(pooled_outputs, 3)
            self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

            # Add highway
            with tf.name_scope("highway"):
                self.h_highway = highway(self.h_pool_flat, self.h_pool_flat.get_shape()[1], 1, 0)

            # Add dropout
            with tf.name_scope("dropout"):
                self.h_drop = tf.nn.dropout(self.h_highway, self.dropout_keep_prob)
            
            
            # Final (unnormalized) scores and predictions
            with tf.name_scope("output"):
                W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
                l2_loss += tf.nn.l2_loss(W)
                l2_loss += tf.nn.l2_loss(b)
                self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
                self.ypred_for_auc = tf.nn.softmax(self.scores)
                self.predictions = tf.argmax(self.scores, 1, name="predictions")
                
            # CalculateMean cross-entropy loss
            with tf.name_scope("loss"):
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
                self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss                
                
            
        self.params = [param for param in tf.trainable_variables() if 'discriminator' in param.name]
        d_optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = d_optimizer.compute_gradients(self.loss, self.params, aggregation_method=2)
        self.train_op = d_optimizer.apply_gradients(grads_and_vars)            

###  Discriminator  Hyper-parameters

In [14]:
#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

vocab_size = 58158

In [15]:
discriminator = Discriminator(sequence_length=217, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim, 
                                filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, l2_reg_lambda=dis_l2_reg_lambda)


### Train the discriminator...

In [16]:
BATCH_SIZE = 64

In [17]:
dis_data_loader = Dis_dataloader(BATCH_SIZE)

In [18]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

## some parameters must change to make sure the model can run: for example sequence length and vocabulary size.


In [20]:
# Train 3 epoch on the generated data and do this for 50 times
for _ in range(3):
    
    negative_file = 'generated_QA.txt'
    positive_file = 'real_QA.txt'
    
    dis_data_loader.load_train_data(positive_file, negative_file)
    
    for _ in range(3):
        print(' epoch: ')
        print(_)
        dis_data_loader.reset_pointer()
        #for it in xrange(dis_data_loader.num_batch):
        for it in range(dis_data_loader.num_batch):
            x_batch, y_batch = dis_data_loader.next_batch()
            feed = {
                discriminator.input_x: x_batch,
                discriminator.input_y: y_batch,
                discriminator.dropout_keep_prob: dis_dropout_keep_prob
            }
            _ = sess.run(discriminator.train_op, feed)

 epoch: 
0
 epoch: 
1
 epoch: 
2
 epoch: 
0
 epoch: 
1
 epoch: 
2
 epoch: 
0
 epoch: 
1
 epoch: 
2
